In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

C:\Users\k1517671\AppData\Local\Continuum\anaconda2\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
from gensim import models

In [3]:
# Plotting tools
import pyLDAvis

In [4]:
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [5]:
#cleaning and filter the words
import datetime
import sys
import codecs
import re
import urllib, urllib2
import itertools, collections

import nltk  # Natural Language Processing
#nltk.download('punkt')
#nltk.download('all')
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords # list of words
from collections import Counter  # optimized way to do this
import string  # list(string.punctuation) - produces a list of punctuations
import copy
from itertools import product, tee, combinations, chain
from nltk.stem import PorterStemmer
from operator import itemgetter # help with dataframes

from scipy.spatial.distance import cosine

from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin
from sklearn.utils import shuffle

#from tweepy import Stream

encodingTot = sys.stdout.encoding or 'utf-8'

In [6]:
# Regex from Gagan ************************************************************

emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""

# Regex_str is used to GET text from CSV file

regex_str = [
    
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-signs
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)' # other words
]

# These Regex are used to EXCLUDE items from the text AFTER IMPORTING from csv with regex_str

numbers = r'(?:(?:\d+,?)+(?:\.?\d+)?)'
URL = r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+'
html_tag = r'<[^>]+>'
hash_tag = r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)"
at_sign = r'(?:@[\w_]+)'
dash_quote = r"(?:[a-z][a-z'\-_]+[a-z])"
other_word = r'(?:[\w_]+)'
other_stuff = r'(?:\S)' # anything else - NOT USED
start_pound = r"([#?])(\w+)" # Start with #
start_quest_pound = r"(?:^|\s)([#?])(\w+)" # Start with ? or with #
cont_number = r'(\w*\d\w*)' # Words containing numbers

# My REGEX **************************************************************************

#      Remove '[' and ']' brackets

sq_br_f = r'(?:[[\w_]+)' # removes '['
sq_br_b = r'(?:][\w_]+)' # removes ']'

rem_bracket = r'(' + '|'.join([sq_br_f, sq_br_b]) +')'
rem_bracketC = re.compile(rem_bracket, re.VERBOSE)

# Removes all words of 3 characters or less *****************************************************

short_words = r'\W*\b\w{1,3}\b' # Short words of 3 character or less
short_wordsC = re.compile(short_words, re.VERBOSE | re.IGNORECASE)

# REGEX remove all words with \ and / combinations

slash_back =  r'\s*(?:[\w_]*\\(?:[\w_]*\\)*[\w_]*)'
slash_fwd = r'\s*(?:[\w_]*/(?:[\w_]*/)*[\w_]*)'
slash_all = r'\s*(?:[\w_]*[/\\](?:[\w_]*[/\\])*[\w_]*)'

# REGEX numbers, short words and URL only to EXCLUDE +++++++++++++++++++++++++++++++++++++++++++++++++++

num_url_short = r'(' + '|'.join([numbers, URL, short_words + sq_br_f + sq_br_b]) +')'  # Exclude from tweets
comp_num_url_short = re.compile(num_url_short, re.VERBOSE | re.IGNORECASE)

# Master REGEX to INCLUDE from the original tweets ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

list_regex = r'(' + '|'.join(regex_str) + ')'

master_regex = re.compile(list_regex, re.VERBOSE | re.IGNORECASE) # TAKE from tweets INITIALLY

In [19]:
char_remove = [']', '[', '(', ')', '{', '}'] # characters to be removed

#the other 3character length words will be removed
words_keep = ['old', 'new', 'age', 'lot', 'bag', 'top', 'cat', 'bat', 'sap', 'jda', 'tea', 'dog', 'lie', 'law', 'lab',\
             'mob', 'map', 'car', 'fat', 'sea', 'saw', 'raw', 'rob', 'win', 'can', 'get', 'fan', 'fun', 'big',\
             'use', 'pea', 'pit','pot', 'pat', 'ear', 'eye', 'kit', 'pot', 'pen', 'bud', 'bet', 'god', 'tax', 'won', 'run',\
              'lid', 'log', 'pr', 'pd', 'cop', 'nyc', 'ny', 'la', 'toy', 'war', 'law', 'lax', 'jfk', 'fed', 'cry', 'ceo',\
              'pay', 'pet', 'fan', 'fun', 'usd', 'rio']

emotion_list = [':)', ';)', '(:', '(;', '}', '{','}']


word_garb = ['here', 'there', 'where', 'when', 'would', 'should', 'could','thats', 'youre', 'thanks', 'hasn',\
             'thank', 'https', 'since', 'wanna', 'gonna', 'aint', 'http', 'unto', 'onto', 'into', 'havent',\
             'dont', 'done', 'cant', 'werent', 'https', 'u', 'isnt', 'go', 'theyre', 'each', 'every', 'shes', 'youve', 'youll',\
            'weve', 'theyve']

extra_word_garb = []

#extra_word_garb = ['today','london','greater']

# Dictionary with Replacement Pairs ******************************************************************************
#repl_dict = {'googleele': 'goog', 'lyin': 'lie', 'googles': 'goog', 'aapl':'apple',\
#             'msft':'microsoft', 'google': 'goog', 'googl':'goog'}

repl_dict = {}

exclude = list(string.punctuation) + emotion_list + word_garb + extra_word_garb

# Convert tuple to a list, then to a string; Remove the characters; Stays as a STRING. Porter Stemmer

stemmer=PorterStemmer()
lmtzr = WordNetLemmatizer()

2. Prerequisites – Download nltk stopwords and spacy model
We will need the stopwords from NLTK and spacy’s en model for text pre-processing. Later, we will be using the spacy model for lemmatization.

Lemmatization is nothing but converting a word to its root word. For example: the lemma of the word ‘machines’ is ‘machine’. Likewise, ‘walking’ –> ‘walk’, ‘mice’ –> ‘mouse’ and so on.

##### Run in python console
import nltk; nltk.download('stopwords')

##### Run in terminal or command prompt
python3 -m spacy download en

## 5. Prepare Stopwords

We have already downloaded the stopwords. Let’s import them and make it available in stop_words.

In [20]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
#stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

## 6. Import Newsgroups Data

We will be using the 20-Newsgroups dataset for this exercise. This version of the dataset contains about 11k newsgroups posts from 20 different topics. This is available as newsgroups.json.

This is imported using pandas.read_json and the resulting dataset has 3 columns as shown.

In [35]:
filename = 'london2017_q2_msoa.csv'

In [36]:
# Import Dataset
directory = 'output/'+ filename
import sys

df = pd.read_csv(open(directory,'rU'), encoding='utf-8', engine='c', header = 0, 
                 error_bad_lines=False, parse_dates=True)
#df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
#print(df.target_names.unique())
df.head()

,index_right,MSOA11CD,MSOA11NM,LAD11CD,LAD11NM,RGN11CD,RGN11NM,USUALRES,HHOLDRES,COMESTRES,POPDEN,HHOLDS,AVHHOLDSZ,text
0,0,E02000001,City of London 001,E09000001,City of London,E12000007,London,7375,7187,188,25.5,4385,1.6,Think like a toddler- when we are at our most ...
1,1,E02000002,Barking and Dagenham 001,E09000002,Barking and Dagenham,E12000007,London,6775,6724,51,31.3,2713,2.5,Just posted a photo @ Moby Dick https://t.co/Z...
2,2,E02000003,Barking and Dagenham 002,E09000002,Barking and Dagenham,E12000007,London,10045,10033,12,46.9,3834,2.6,Working @ Chadwell Heath https://t.co/STy2bMBJ...
3,3,E02000004,Barking and Dagenham 003,E09000002,Barking and Dagenham,E12000007,London,6182,5937,245,24.8,2318,2.6,#Shielding🛡🤓🎸 @ Dagenham United Kingdom ht...
4,4,E02000005,Barking and Dagenham 004,E09000002,Barking and Dagenham,E12000007,London,8562,8562,0,72.1,3183,2.7,Kye's in the pub for the first time. Ordered h...


## 7. Remove hashtag, URL, and at_sign characters
As you can see there are many emails, newline and extra spaces that is quite distracting. Let’s get rid of them using regular expressions.

In [112]:
def clean_doc(filename):
    # Convert to list
    directory = 'output/'+ filename
    import sys

    df = pd.read_csv(open(directory,'rU'), encoding='utf-8', engine='c', header = 0, 
                 error_bad_lines=False, parse_dates=True)
    tweet_list_org = df.text.values.tolist()
    tweet_list_org = [str(l).encode('ascii','ignore') for l in tweet_list_org]
    outlist_init = filterPick(tweet_list_org, search_regex) # It is a tuple: initial list from all tweets
    tweet_clean_fin = [] # Cleaned Tweets - Final Version
    # Preparing CLEAN tweets tp keep SEPARATELY from WORDS in TWEETS
    count = 0
    t0 = time()
    print('starting time is %s' , t0)
    for tweet in outlist_init:
        count = count +1
        if count%100 == 0:
            print(count)
            print("done in %0.3fs." % (time() - t0))
        tw_clean = []
        tw_clean = [ch for ch in tweet if ch not in char_remove]

        tw_clean = re.sub(URL, "", str(tw_clean))
        tw_clean = re.sub(html_tag, "",str(tw_clean))
        tw_clean = re.sub(hash_tag, "",str(tw_clean))
        tw_clean = re.sub(slash_all,"", str(tw_clean))
        tw_clean = re.sub(cont_number, "",str(tw_clean))
        tw_clean = re.sub(numbers, "",str(tw_clean))
        tw_clean = re.sub(start_pound, "",str(tw_clean))
        tw_clean = re.sub(start_quest_pound, "",str(tw_clean))
        tw_clean = re.sub(at_sign, "",str(tw_clean))
        tw_clean = re.sub("'", "",str(tw_clean))
        tw_clean = re.sub('"', "",str(tw_clean))
        tw_clean = re.sub(r'(?:^|\s)[@#].*?(?=[,;:.!?]|\s|$)', r'', tw_clean) # Removes # and @ in words (lookahead)
        tw_clean = lmtzr.lemmatize(str(tw_clean))
        #tw_clean = stemmer.stem(str(tw_clean))
    
        tw_clean_lst = re.findall(r'\w+', str(tw_clean))
    
        tw_clean_lst = [tw.lower() for tw in tw_clean_lst if tw.lower() not in stopwords.words('english')]
        tw_clean_lst = [word for word in tw_clean_lst if word not in exclude]
        tw_clean_lst = str([word for word in tw_clean_lst if len(word)>3 or word.lower() in words_keep])
        
        tw_clean_lst = re.findall(r'\w+', str(tw_clean_lst))
        #tw_clean_lst = [replace_all(word, repl_dict) for word in tw_clean_lst]
        
        tweet_clean_fin.append(list(tw_clean_lst))
    data = [" ".join(i) for i in tweet_clean_fin]
    data = pd.DataFrame(data)
    to_filename = 'cleaned_'+ filename
    to_filename
    data.to_csv(to_filename, encoding='utf-8' )

In [ ]:
filenames = ['london2017_q2_msoa.csv','london2017_q1_msoa.csv','london2016_q4_msoa.csv','london2016_q3_msoa.csv',
             'london2016_q2_msoa.csv','london2016_q1_msoa.csv','london2015_q4_msoa.csv','london2015_q3_msoa.csv']

for filename in filenames[1:]:
    clean_doc(filename)
    

(u'starting time is %s', 1530297493.486)


In [37]:
# Convert to list
data = df.text.values.tolist()
#data[:1]

In [38]:
# Filters IMPORTED from csv file data

def filterPick(list, filter):
    return [ ( l, m.group(1) ) for l in list for m in (filter(l),) if m]

search_regex = re.compile(list_regex, re.VERBOSE | re.IGNORECASE).search

import sys
stdi,stdo,stde=sys.stdin,sys.stdout,sys.stderr
reload(sys)
sys.setdefaultencoding('utf8')
sys.stdin,sys.stdout,sys.stderr=stdi,stdo,stde
print (sys.stdout)

# Convert to list
tweet_list_org = df.text.values.tolist()

tweet_list_org = [str(l).encode('ascii','ignore') for l in tweet_list_org]

outlist_init = filterPick(tweet_list_org, search_regex) # It is a tuple: initial list from all tweets

In [39]:
from time import time

In [40]:
tweet_clean_fin = [] # Cleaned Tweets - Final Version
# Preparing CLEAN tweets tp keep SEPARATELY from WORDS in TWEETS
count = 0
t0 = time()
print('starting time is %s' , t0)

for tweet in outlist_init:
    count = count +1
    if count%100 == 0:
        print(count)
        print("done in %0.3fs." % (time() - t0))
    tw_clean = []
    tw_clean = [ch for ch in tweet if ch not in char_remove]

    tw_clean = re.sub(URL, "", str(tw_clean))
    tw_clean = re.sub(html_tag, "",str(tw_clean))
    tw_clean = re.sub(hash_tag, "",str(tw_clean))
    tw_clean = re.sub(slash_all,"", str(tw_clean))
    tw_clean = re.sub(cont_number, "",str(tw_clean))
    tw_clean = re.sub(numbers, "",str(tw_clean))
    tw_clean = re.sub(start_pound, "",str(tw_clean))
    tw_clean = re.sub(start_quest_pound, "",str(tw_clean))
    tw_clean = re.sub(at_sign, "",str(tw_clean))
    tw_clean = re.sub("'", "",str(tw_clean))
    tw_clean = re.sub('"', "",str(tw_clean))
    tw_clean = re.sub(r'(?:^|\s)[@#].*?(?=[,;:.!?]|\s|$)', r'', tw_clean) # Removes # and @ in words (lookahead)

    tw_clean = lmtzr.lemmatize(str(tw_clean))
    #tw_clean = stemmer.stem(str(tw_clean))
    
    tw_clean_lst = re.findall(r'\w+', str(tw_clean))
    
    tw_clean_lst = [tw.lower() for tw in tw_clean_lst if tw.lower() not in stopwords.words('english')]
    tw_clean_lst = [word for word in tw_clean_lst if word not in exclude]
    tw_clean_lst = str([word for word in tw_clean_lst if len(word)>3 or word.lower() in words_keep])
    
    tw_clean_lst = re.findall(r'\w+', str(tw_clean_lst))
    #tw_clean_lst = [replace_all(word, repl_dict) for word in tw_clean_lst]
    
    tweet_clean_fin.append(list(tw_clean_lst))

(u'starting time is %s', 1530289287.348)
100
done in 147.789s.
200
done in 417.159s.
300
done in 536.266s.
400
done in 778.709s.
500
done in 849.330s.
600
done in 1159.556s.
700
done in 1290.224s.
800
done in 1496.084s.
900
done in 1705.421s.


In [41]:
# Remove Emails
#data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
#data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
#data = [re.sub("\'", "", sent) for sent in data]

data = [" ".join(i) for i in tweet_clean_fin]

pprint(data[:1])

[u'think like toddler essential vital curious joyful playful happy discover design exhibited next award happy discover design happy discover design beautiful view students sample work academy today used create garden gyalchester sushisamba floor shard great view londons iconic buildings image captured another mind blowing views londons exclusive observation decks open terrace city thameslink railway station greater london london guys clueless lilwayneofficial_ eminem train coming venues barbican students sample work academy today used years still going strong kanaloa club stations impressive spring skies royal exchange grand sauterelle big shout billingsgate market beautiful week whos coming ride morning carliwheatley team shop stone marble case available phone models free shipping shop stone marble case available phone models free shipping definitely getting best photography last nights cocktails byoceast deserve flash mob london london early birthday celebrations stunner aesthetic cu

]


In [43]:
data = pd.DataFrame(data)

In [44]:
to_filename = 'cleaned_'+ filename
to_filename
data.to_csv(to_filename, encoding='utf-8' )

In [46]:
ori_data = data

After removing the emails and extra spaces, the text still looks messy. It is not ready for the LDA to consume. You need to break down each sentence into a list of words through tokenization, while clearing up all the messy text in the process.

Gensim’s simple_preprocess is great for this.

## 8. Tokenize words and Clean-up text

Let’s tokenize each sentence into a list of words, removing punctuations and unnecessary characters altogether.
Gensim’s simple_preprocess() is great for this. Additionally I have set deacc=True to remove the punctuations.

In [45]:
import sys
stdi,stdo,stde=sys.stdin,sys.stdout,sys.stderr
reload(sys)
sys.setdefaultencoding('utf8')
sys.stdin,sys.stdout,sys.stderr=stdi,stdo,stde
print (sys.stdout)

In [47]:
data = data.values.tolist()

In [48]:
data[1]

[u'posted photo moby dick limbo contest going tantony green limbo wheelies slow races went well today help great fun kids foot fone wardour street guest speaker adoptionuk evening romford talking advocacy education tonight wardour street bloody weekend ibiza theme fest premier quick visits farm summer open often mcfilth mcdonalds final part malaysia vlog series']

In [49]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[[u'think', u'like', u'toddler', u'essential', u'vital', u'curious', u'joyful', u'playful', u'happy', u'discover', u'design', u'exhibited', u'next', u'award', u'happy', u'discover', u'design', u'happy', u'discover', u'design', u'beautiful', u'view', u'students', u'sample', u'work', u'academy', u'today', u'used', u'create', u'garden', u'gyalchester', u'sushisamba', u'floor', u'shard', u'great', u'view', u'londons', u'iconic', u'buildings', u'image', u'captured', u'another', u'mind', u'blowing', u'views', u'londons', u'exclusive', u'observation', u'decks', u'open', u'terrace', u'city', u'thameslink', u'railway', u'station', u'greater', u'london', u'london', u'guys', u'clueless', u'eminem', u'train', u'coming', u'venues', u'barbican', u'students', u'sample', u'work', u'academy', u'today', u'used', u'years', u'still', u'going', u'strong', u'kanaloa', u'club', u'stations', u'impressive', u'spring', u'skies', u'royal', u'exchange', u'grand', u'sauterelle', u'big', u'shout', u'billingsgate', 

## 9. Creating Bigram and Trigram Models (this is very important)
Bigrams are two words frequently occurring together in the document. Trigrams are 3 words frequently occurring.

Some examples in our example are: ‘front_bumper’, ‘oil_leak’, ‘maryland_college_park’ etc.

Gensim’s Phrases model can build and implement the bigrams, trigrams, quadgrams and more. The two important arguments to Phrases are min_count and threshold. The higher the values of these param, the harder it is for words to be combined to bigrams.

In [60]:
#Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[:1]]])

## 10. Remove Stopwords, Make Bigrams and Lemmatize

The bigrams model is ready. Let’s define the functions to remove the stopwords, make bigrams and lemmatization and call them sequentially.

In [61]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]



In [62]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

In [63]:
from __future__ import unicode_literals
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
import en_core_web_sm
nlp = en_core_web_sm.load(disable=['parser', 'ner'])

#nlp = spacy.load('en', disable=['parser', 'ner'])

In [64]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [65]:

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[[u'think', u'toddler', u'essential', u'vital', u'curious', u'joyful', u'playful', u'happy', u'discover', u'design', u'exhibit', u'next', u'award', u'happy', u'discover', u'design', u'happy', u'discover', u'design', u'beautiful', u'view', u'student', u'sample', u'work', u'academy', u'today', u'use', u'create', u'garden', u'gyalchester', u'sushisamba_floor', u'shard', u'great', u'view', u'london', u'iconic', u'building', u'image', u'capture', u'mind_blowing', u'view', u'london', u'exclusive', u'observation', u'deck', u'open', u'terrace', u'city', u'thameslink', u'railway_station', u'great', u'london', u'london', u'guy', u'clueless', u'eminem', u'train', u'come', u'venue', u'barbican', u'student', u'sample', u'work', u'academy', u'today', u'use', u'year', u'still', u'go', u'strong', u'kanaloa_club', u'station', u'impressive', u'spring', u'sky', u'royal_exchange', u'grand_sauterelle', u'big', u'shout', u'billingsgate', u'market', u'beautiful', u'week', u'who', u's', u'come', u'ride', u'mo

## 11. Create the Dictionary and Corpus needed for Topic Modeling
The two main inputs to the LDA topic model are the dictionary(id2word) and the corpus. Let’s create them.

In [66]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 5), (3, 1), (4, 2), (5, 2), (6, 3), (7, 1), (8, 1), (9, 5), (10, 1), (11, 1), (12, 1), (13, 7), (14, 1), (15, 2), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 2), (22, 15), (23, 2), (24, 5), (25, 1), (26, 6), (27, 1), (28, 2), (29, 1), (30, 1), (31, 9), (32, 4), (33, 2), (34, 6), (35, 3), (36, 8), (37, 1), (38, 1), (39, 2), (40, 1), (41, 1), (42, 3), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 4), (49, 1), (50, 17), (51, 1), (52, 1), (53, 2), (54, 1), (55, 1), (56, 2), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 16), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 17), (71, 1), (72, 1), (73, 1), (74, 1), (75, 2), (76, 1), (77, 3), (78, 4), (79, 2), (80, 3), (81, 22), (82, 1), (83, 1), (84, 8), (85, 1), (86, 1), (87, 1), (88, 2), (89, 1), (90, 1), (91, 1), (92, 71), (93, 1), (94, 13), (95, 1), (96, 1), (97, 3), (98, 6), (99, 14), (100, 4), (101, 4), (102, 9), (103, 5), (104, 1), (105, 22), (106, 2), (107, 1), (108, 1), (109, 5),

Gensim creates a unique id for each word in the document. The produced corpus shown above is a mapping of (word_id, word_frequency).

For example, (0, 1) above implies, word id 0 occurs once in the first document. Likewise, word id 1 occurs twice and so on.

This is used as the input by the LDA model.

If you want to see what word a given id corresponds to, pass the id as a key to the dictionary.

In [67]:
id2word[0]

u'thousand_word'

In [68]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[(u'thousand_word', 1),
  (u'somerset_house', 1),
  (u'yellow', 5),
  (u'railing', 1),
  (u'gabriella', 2),
  (u'cyprus', 2),
  (u'otro', 3),
  (u'electricity', 1),
  (u'otra', 1),
  (u'lord', 5),
  (u'lora', 1),
  (u'inclement', 1),
  (u'hormone', 1),
  (u'cipfa', 7),
  (u'bruschetta', 1),
  (u'deli', 2),
  (u'capoeira', 1),
  (u'hunni', 1),
  (u'fur', 1),
  (u'scholar', 1),
  (u'wooden', 1),
  (u'showcas', 2),
  (u'wednesday', 15),
  (u'tambin', 2),
  (u'solid', 5),
  (u'oooh', 1),
  (u'straight', 6),
  (u'fritter', 1),
  (u'charter', 2),
  (u'douard', 1),
  (u'jamjam', 1),
  (u'tired', 9),
  (u'miller', 4),
  (u'hanging', 2),
  (u'bacon', 6),
  (u'elegant', 3),
  (u'second', 8),
  (u'agethangelou', 1),
  (u'rehaut', 1),
  (u'admire', 2),
  (u'brexit', 1),
  (u'ruthless', 1),
  (u'cooking', 3),
  (u'lariojaaltasa', 1),
  (u'piala', 1),
  (u'fossil', 1),
  (u'hero', 1),
  (u'jani_london', 1),
  (u'herb', 4),
  (u'hydrangea', 1),
  (u'here', 17),
  (u'leathercraft', 1),
  (u'lgbt', 1)

## 12. Building the Topic Model
We have everything required to train the LDA model. In addition to the corpus and dictionary, you need to provide the number of topics as well.

Apart from that, alpha and eta are hyperparameters that affect sparsity of the topics. According to the Gensim docs, both defaults to 1.0/num_topics prior.

chunksize is the number of documents to be used in each training chunk. update_every determines how often the model parameters should be updated and passes is the total number of training passes.

In [69]:
# Build LDA model
lda_model_20_100 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

KeyboardInterrupt: 

In [ ]:
# Build LDA model
lda_model_10_100 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

## 13. View the topics in LDA model

The above LDA model is built with 20 different topics where each topic is a combination of keywords and each keyword contributes a certain weightage to the topic.You can see the keywords for each topic and the weightage(importance) of each keyword using lda_model.print_topics() as shown next.

In [70]:
# Print the Keyword in the 10 topics
pprint(lda_model_20_100.print_topics())
doc_lda = lda_model_20_100[corpus]

[(0,
  u'0.008*"richmond" + 0.007*"garden" + 0.007*"view" + 0.006*"park_lane" + 0.006*"city" + 0.005*"brick_lane" + 0.005*"herne_hill" + 0.005*"sutton" + 0.004*"opera" + 0.004*"love"'),
 (1,
  u'0.017*"wharf" + 0.012*"knightsbridge" + 0.010*"quay" + 0.010*"battersea_park" + 0.009*"rose_garden" + 0.009*"hampton_court" + 0.008*"chelsea" + 0.008*"hall" + 0.008*"albert_memorial" + 0.007*"city_airport"'),
 (2,
  u'0.012*"greenwich_greenwich" + 0.010*"houses_parliament" + 0.010*"piccadilly_circus" + 0.009*"leyton" + 0.009*"lewisham" + 0.009*"repostby" + 0.006*"johns_wood" + 0.005*"dalston" + 0.004*"londra" + 0.004*"lion_king"'),
 (3,
  u'0.033*"trafalgar_square" + 0.025*"battersea" + 0.012*"queen_mary" + 0.006*"highbury" + 0.006*"ghost" + 0.005*"stockwell" + 0.005*"ridiculous" + 0.005*"nobu" + 0.005*"jamess_park" + 0.005*"shepherds_bush"'),
 (4,
  u'0.008*"love" + 0.008*"good" + 0.007*"get" + 0.007*"new" + 0.007*"time" + 0.007*"posted_photo" + 0.006*"great" + 0.005*"make" + 0.005*"work" + 0.

How to interpret this?

Topic 0 is a represented as _0.016“car” + 0.014“power” + 0.010“light” + 0.009“drive” + 0.007“mount” + 0.007“controller” + 0.007“cool” + 0.007“engine” + 0.007“back” + ‘0.006“turn”.

It means the top 10 keywords that contribute to this topic are: ‘car’, ‘power’, ‘light’.. and so on and the weight of ‘car’ on topic 0 is 0.016.

The weights reflect how important a keyword is to that topic.

Looking at these keywords, can you guess what this topic could be? You may summarise it either are ‘cars’ or ‘automobiles’.

Likewise, can you go through the remaining topic keywords and judge what the topic is?

## 14. Compute Model Perplexity and Coherence Score

Model perplexity and topic coherence provide a convenient measure to judge how good a given topic model is. In my experience, topic coherence score, in particular, has been more helpful.

In [71]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

(u'\nPerplexity: ', -9.2930347183478013)
(u'\nCoherence Score: ', 0.55026083521977009)


In [75]:
# Compute Perplexity
print('\nPerplexity: ', lda_model_10_100.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model_10_100, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

(u'\nPerplexity: ', -9.2814889907915301)
(u'\nCoherence Score: ', 0.55862864700861603)


## 15. Visualize the topics-keywords
Now that the LDA model is built, the next step is to examine the produced topics and the associated keywords. There is no better tool than pyLDAvis package’s interactive chart and is designed to work well with jupyter notebooks.

In [72]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
4      38.746827        1       1  0.371846 -0.006503
13     17.171645        1       2  0.333612  0.005572
7      15.540476        1       3  0.307688 -0.001962
14      4.557583        1       4  0.219384  0.002921
11      3.853447        1       5  0.140776 -0.010025
0       3.442442        1       6  0.144533  0.010739
10      2.824904        1       7  0.054149  0.022374
12      2.698210        1       8 -0.002173  0.077647
19      1.463157        1       9 -0.099853 -0.400684
2       1.251273        1      10 -0.141279  0.032089
16      1.137316        1      11 -0.109135  0.031632
1       1.123521        1      12 -0.108953  0.004025
17      1.085040        1      13 -0.123559  0.021581
5       1.034448        1      14 -0.129331  0.034593
15      0.926012        1      15 -0.132942  0.026376
3       0.913504        1      16 -0.136737  0.035649
9       0.774927        1      17 -0.135579  0.029618
6       0.626829        1      18 -0.145596  0.030277
8       0.502074        1      19 -0.154466  0.029125
18      0.326367        1      20 -0.152385  0.024955, topic_info=      Category         Freq                 Term        Total  loglift  \
term                                                                     
7565   Default  1397.000000              station  1397.000000  30.0000   
51071  Default  1273.000000                 park  1273.000000  29.0000   
54818  Default  1512.000000                 soho  1512.000000  28.0000   
27227  Default   615.000000        covent_garden   615.000000  27.0000   
22736  Default  1472.000000               garden  1472.000000  26.0000   
68654  Default   462.000000                 mile   462.000000  25.0000   
53940  Default   781.000000            greenwich   781.000000  24.0000   
55880  Default  1109.000000                hotel  1109.000000  23.0000   
48895  Default  1174.000000               studio  1174.000000  22.0000   
50749  Default   581.000000                  eye   581.000000  21.0000   
10778  Default   604.000000      railway_station   604.000000  20.0000   
40031  Default   884.000000                 city   884.000000  19.0000   
1878   Default   374.000000     leicester_square   374.000000  18.0000   
8941   Default  2368.000000           last_night  2368.000000  17.0000   
68717  Default   837.000000       happy_birthday   837.000000  16.0000   
63299  Default   342.000000    buckingham_palace   342.000000  15.0000   
2581   Default   481.000000           marylebone   481.000000  14.0000   
76549  Default  1186.000000                 view  1186.000000  13.0000   
9247   Default  1182.000000                 home  1182.000000  12.0000   
19780  Default   318.000000           paddington   318.000000  11.0000   
5680   Default  2300.000000                night  2300.000000  10.0000   
33747  Default  1472.000000               street  1472.000000   9.0000   
28880  Default  4076.000000                  new  4076.000000   8.0000   
42424  Default   292.000000     trafalgar_square   292.000000   7.0000   
69481  Default  3576.000000         posted_photo  3576.000000   6.0000   
18982  Default   582.000000                 cake   582.000000   5.0000   
4078   Default  3033.000000              tonight  3033.000000   4.0000   
49497  Default   355.000000       united_kingdom   355.000000   3.0000   
20396  Default  4993.000000                 love  4993.000000   2.0000   
16441  Default   454.000000                tower   454.000000   1.0000   
...        ...          ...                  ...          ...      ...   
75816  Topic20    14.138394                turie    14.722428   5.6844   
33559  Topic20    13.400512  southwark_playhouse    13.984547   5.6822   
56535  Topic20    13.208689        delivery_para    13.792724   5.6816   
40898  Topic20    13.208689       meninas_compre    13.792724   5.6816   
75736  Topic20    13.208689    

In [76]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_10_100, corpus, id2word)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      57.485449        1       1  0.337507  0.006524
4      14.410044        1       2  0.287431 -0.036277
3      12.650831        1       3  0.271646 -0.028205
9       3.965216        1       4 -0.069416  0.324869
7       3.195310        1       5 -0.122046  0.119609
5       1.976284        1       6 -0.144399 -0.036161
0       1.886900        1       7 -0.117377 -0.116817
6       1.852569        1       8 -0.132670 -0.095849
8       1.526959        1       9 -0.156947 -0.067935
1       1.050437        1      10 -0.153729 -0.069757, topic_info=      Category         Freq                Term        Total  loglift  logprob
term                                                                          
54818  Default  1274.000000                soho  1274.000000  30.0000  30.0000
27227  Default   787.000000       covent_garden   787.000000  29.0000  29.0000
53940  Default   530.000000           greenwich   530.000000  28.0000  28.0000
51071  Default  1293.000000                park  1293.000000  27.0000  27.0000
7565   Default  1252.000000             station  1252.000000  26.0000  26.0000
40031  Default   856.000000                city   856.000000  25.0000  25.0000
55880  Default  1114.000000               hotel  1114.000000  24.0000  24.0000
69231  Default   849.000000             theatre   849.000000  23.0000  23.0000
33747  Default  1591.000000              street  1591.000000  22.0000  22.0000
28880  Default  4059.000000                 new  4059.000000  21.0000  21.0000
11303  Default  3604.000000                 get  3604.000000  20.0000  20.0000
2581   Default   475.000000          marylebone   475.000000  19.0000  19.0000
16441  Default   457.000000               tower   457.000000  18.0000  18.0000
1878   Default   443.000000    leicester_square   443.000000  17.0000  17.0000
4078   Default  3059.000000             tonight  3059.000000  16.0000  16.0000
12571  Default  1154.000000          restaurant  1154.000000  15.0000  15.0000
74215  Default  3138.000000                come  3138.000000  14.0000  14.0000
73744  Default  1104.000000            cocktail  1104.000000  13.0000  13.0000
67388  Default   400.000000         westminster   400.000000  12.0000  12.0000
47084  Default   303.000000        canary_wharf   303.000000  11.0000  11.0000
16680  Default   433.000000             mayfair   433.000000  10.0000  10.0000
11325  Default  1979.000000             morning  1979.000000   9.0000   9.0000
22736  Default  1407.000000              garden  1407.000000   8.0000   8.0000
62814  Default  3157.000000                work  3157.000000   7.0000   7.0000
24512  Default  3285.000000                make  3285.000000   6.0000   6.0000
10778  Default   558.000000     railway_station   558.000000   5.0000   5.0000
70216  Default  4755.000000                good  4755.000000   4.0000   4.0000
55960  Default   275.000000         marble_arch   275.000000   3.0000   3.0000
38784  Default   314.000000       oxford_circus   314.000000   2.0000   2.0000
60509  Default   353.000000           hyde_park   353.000000   1.0000   1.0000
...        ...          ...                 ...          ...      ...      ...
23772  Topic10    32.276577             whitton    32.827760   4.5390  -5.7385
62745  Topic10    31.436301          haus_silva    31.987498   4.5386  -5.7649
23798  Topic10    30.933053        burgess_park    31.485030   4.5383  -5.7810
51671  Topic10    28.247795    twickenham_stoop    28.798995   4.5366  -5.8718
54032  Topic10    27.482152          blackheath    28.033318   4.5361  -5.8993
37328  Topic10    26.233383  puregym_bermondsey    26.784622   4.5352  -5.9458
6423   Topic10    25.132047          roman_road    25.683522   4.5343  -5.9887
65646  Topic10    24.460738          white_post    25.011976   4.5337  -6.0158
7570   Topic10    22.622941         sager_wilde    23.178932   4.5317 

In [79]:


# Create Document - Topic Matrix
data_vectorized = tf_vectorizer.fit_transform(data_lemmatized)
lda_output = lda_model_10_100.transform(data_vectorized)

NameError: name 'tf_vectorizer' is not defined

In [78]:
# Create Document - Topic Matrix
data_vectorized = tf_vectorizer.fit_transform(data_lemmatized)
lda_output = lda_model_10_100.transform(data_vectorized)

# index names
docnames = ["Doc" + str(i) for i in range(len(data_lemmatized))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

So how to infer pyLDAvis’s output?

Each bubble on the left-hand side plot represents a topic. The larger the bubble, the more prevalent is that topic.

A good topic model will have fairly big, non-overlapping bubbles scattered throughout the chart instead of being clustered in one quadrant.

A model with too many topics, will typically have many overlaps, small sized bubbles clustered in one region of the chart.

Alright, if you move the cursor over one of the bubbles, the words and bars on the right-hand side will update. These words are the salient keywords that form the selected topic.

We have successfully built a good looking topic model.

Given our prior knowledge of the number of natural topics in the document, finding the best model was fairly straightforward.

Upnext, we will improve upon this model by using Mallet’s version of LDA algorithm and then we will focus on how to arrive at the optimal number of topics given any large corpus of text.

## 16. Building LDA Mallet Model

So far you have seen Gensim’s inbuilt version of the LDA algorithm. Mallet’s version, however, often gives a better quality of topics.

Gensim provides a wrapper to implement Mallet’s LDA from within Gensim itself. You only need to download the zipfile, unzip it and provide the path to mallet in the unzipped directory to gensim.models.wrappers.LdaMallet. See how I have done this below.

https://anoopacademia.wordpress.com/2015/01/18/install-mallet-on-windows-operating-system/

In [98]:
#import tempfile
#tempfile.tempdir='F:/'

import os 
os.environ.update({'MALLET_HOME':r'F:/workspace/python/mallet-2.0.8/'})


In [99]:
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
mallet_path = 'F:/workspace/python/mallet-2.0.8/bin/mallet' # update this path

In [100]:
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

In [103]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

[(11,
  [(u'camden', 0.02592945),
   (u'market', 0.015425334),
   (u'kingston', 0.01331624),
   (u'night', 0.012654563),
   (u'good', 0.012199661),
   (u'time', 0.010421406),
   (u'great', 0.010297341),
   (u'love', 0.010131922),
   (u'today', 0.009677019),
   (u'thame', 0.009098052)]),
 (18,
  [(u'london', 0.021681523),
   (u'terminal_hounslow', 0.021488799),
   (u'hounslow', 0.019850638),
   (u'lounge', 0.018019754),
   (u'brixton', 0.0152734285),
   (u'heathrow', 0.0137316305),
   (u'great', 0.012912551),
   (u'time', 0.011129849),
   (u'good', 0.010696217),
   (u'home', 0.010166225)]),
 (17,
  [(u'garden', 0.014276523),
   (u'house', 0.0102370735),
   (u'beautiful', 0.009308651),
   (u'morning', 0.009210923),
   (u'lovely', 0.008526823),
   (u'walk', 0.00781829),
   (u'place', 0.007728705),
   (u'find', 0.007696129),
   (u'life', 0.0071341894),
   (u'start', 0.006466377)]),
 (15,
  [(u'today', 0.015963824),
   (u'session', 0.011816728),
   (u'game', 0.010317777),
   (u'croydon', 0.

## 17. How to find the optimal number of topics for LDA?

My approach to finding the optimal number of topics is to build many LDA models with different values of number of topics (k) and pick the one that gives the highest coherence value.

Choosing a ‘k’ that marks the end of a rapid growth of topic coherence usually offers meaningful and interpretable topics. Picking an even higher value can sometimes provide more granular sub-topics.

If you see the same keywords being repeated in multiple topics, it’s probably a sign that the ‘k’ is too large.

The compute_coherence_values() (see below) trains multiple LDA models and provides the models and their corresponding coherence scores.

In [104]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [111]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=5, limit=10, step=1)

KeyboardInterrupt: 

In [ ]:
# Show graph
limit=40; start=5; step=1;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [107]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

(u'Num Topics =', 2, u' has Coherence Value of', 0.798)
(u'Num Topics =', 8, u' has Coherence Value of', 0.7245)
(u'Num Topics =', 14, u' has Coherence Value of', 0.7084)
(u'Num Topics =', 20, u' has Coherence Value of', 0.6806)
(u'Num Topics =', 26, u' has Coherence Value of', 0.668)
(u'Num Topics =', 32, u' has Coherence Value of', 0.6398)
(u'Num Topics =', 38, u' has Coherence Value of', 0.6545)


If the coherence score seems to keep increasing, it may make better sense to pick the model that gave the highest CV before flattening out. This is exactly the case here.

So for further steps I will choose the model with 20 topics itself.

In [108]:
# Select the model and print the topics
optimal_model = model_list[3]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

[(0,
  u'0.017*"today" + 0.012*"week" + 0.011*"session" + 0.010*"back" + 0.010*"work" + 0.010*"class" + 0.009*"start" + 0.009*"good" + 0.009*"morning" + 0.008*"drinking"'),
 (1,
  u'0.013*"battersea" + 0.012*"today" + 0.010*"club" + 0.009*"love" + 0.009*"holborn" + 0.009*"good" + 0.008*"show" + 0.008*"putney" + 0.008*"time" + 0.008*"covent_garden"'),
 (2,
  u'0.126*"london" + 0.055*"great" + 0.043*"soho" + 0.012*"street" + 0.012*"night" + 0.009*"love" + 0.009*"club" + 0.008*"time" + 0.008*"good" + 0.007*"theatre"'),
 (3,
  u'0.023*"east" + 0.020*"stratford" + 0.013*"canary_wharf" + 0.011*"posted_photo" + 0.010*"city" + 0.009*"great" + 0.008*"good" + 0.008*"islington" + 0.008*"time" + 0.008*"love"'),
 (4,
  u'0.018*"night" + 0.017*"covent_garden" + 0.012*"great" + 0.011*"theatre" + 0.010*"leicester_square" + 0.009*"good" + 0.009*"birthday" + 0.009*"love" + 0.008*"happy" + 0.008*"posted_photo"'),
 (5,
  u'0.017*"game" + 0.012*"twickenham" + 0.011*"today" + 0.011*"time" + 0.011*"stadium" 

## 18. Finding the dominant topic in each sentence

One of the practical application of topic modeling is to determine what topic a given document is about.

To find that, we find the topic number that has the highest percentage contribution in that document.

The format_topics_sentences() function below nicely aggregates this information in a presentable table.

In [109]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

NameError: name 'lda_model' is not defined

## 19. Find the most representative document for each topic

Sometimes just the topic keywords may not be enough to make sense of what a topic is about. So, to help with understanding the topic, you can find the documents a given topic has contributed to the most and infer the topic by reading that document. Whew!!

In [ ]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head()

The tabular output above actually has 20 rows, one each for a topic. It has the topic number, the keywords, and the most representative document. The Perc_Contribution column is nothing but the percentage contribution of the topic in the given document.



## 20. Topic distribution across documents

Finally, we want to understand the volume and distribution of topics in order to judge how widely it was discussed. The below table exposes that information.

In [ ]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics

## 21. Conclusion

We started with understanding what topic modeling can do. We built a basic topic model using Gensim’s LDA and visualize the topics using pyLDAvis. Then we built mallet’s LDA implementation. You saw how to find the optimal number of topics using coherence scores and how you can come to a logical understanding of how to choose the optimal model.

Finally we saw how to aggregate and present the results to generate insights that may be in a more actionable.

Hope you enjoyed reading this. I would appreciate if you leave your thoughts in the comments section below.